In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from omegaconf import OmegaConf
from sklearn.model_selection import train_test_split

In [3]:
cfg = OmegaConf.load('../config.yaml')

In [9]:
processed_data_dir = '..' / pathlib.Path(cfg.paths.data.processed)
processed_file = processed_data_dir / cfg.files.processed_dataset

df = pd.read_csv(processed_file)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38377 entries, 0 to 38376
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lat              33530 non-null  float64
 1   long             33530 non-null  float64
 2   download_kbit    38377 non-null  float64
 3   upload_kbit      38377 non-null  float64
 4   ping_ms          38377 non-null  float64
 5   lte_rsrp         24675 non-null  float64
 6   signal_strength  3382 non-null   float64
 7   hour             38377 non-null  float64
 8   platform         38371 non-null  object 
 9   network_name     25176 non-null  object 
 10  hour_cat         38035 non-null  object 
 11  cat_technology   38377 non-null  object 
dtypes: float64(8), object(4)
memory usage: 3.5+ MB


## Train Test Split